In [1]:
# import statements
import utils
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [2]:
# Loading data

# Fake data
articles = np.asarray([
    ["Donald Trump buys a dog", "Donald Trump recently was found buying a dog"],
    ["President Trump gets a corgie", "The White House welcomes a new furry friend, and its name is Evan!"],
    ["U.S. Presidential Paws", "You won't believe the new dog that Donald and the fam just got!"],
    ["Biden sells his old car", "President Biden just announced that he is selling is '96 Civic"],
    ["President Biden to sell car", "Biden announced via Twitter that he will indeed be selling his car"],
    ["Biden getting that car cash", "Biden is about to cash out once he sells that junker civic!"]
])

data = utils.toframe(utils.files)

Empty DataFrame
Columns: [article_id, title, abstract, author, source, time_stamp, link, query]
Index: []


In [3]:
data.head()

,article_id,title,abstract,author,source,time_stamp,link,query


In [4]:
# Vectorizing 
roberta = SentenceTransformer('stsb-roberta-base')
a = [x + ': ' + y for x,y in zip(data['title'], data['abstract'])]
v = utils.encode(a, roberta)

Time for computting embeddings:0.017902135848999023


In [ ]:
# Retrieving related articles 

In [ ]:
# Creating the graph
import networkx as nx

G = nx.Graph()

In [ ]:
G.clear()
for root, v0 in zip(data['title'], v):
    for comp, v1 in zip(data['title'], v):
        if utils.doc_sim(v0,v1) > .5:
            G.add_edge(root, comp, weight=utils.doc_sim(v0,v1))

In [ ]:
nx.draw(G)

In [ ]:
from networkx.algorithms import community
from networkx import edge_betweenness_centrality as betweenness

def most_central_edge(G):

    centrality = betweenness(G, weight="weight")

    return max(centrality, key=centrality.get)

comp = community.girvan_newman(G, most_valuable_edge=most_central_edge)

communities =tuple(sorted(c) for c in next(comp))

In [ ]:
# Extracting full articles from communities
cdf = []
for com in communities:
    df = pd.DataFrame()
    for art in com:
        x = data.loc[data['title'] == art]
        df = df.append(x)
    cdf.append(df)

In [ ]:
# Reducing each community down to an article 
min_coms = [] 
for com in cdf:
    min_time = min(com['time_stamp'])
    df = pd.DataFrame()
    df = df.append(com.loc[com['time_stamp'] == min_time])
    min_coms.append(df)

In [ ]:
el = nx.Graph()
el.clear()

# Generating Lv and Huang graph 
lv = sorted([(x,list(x['time_stamp'])) for x in min_coms], key = lambda x: x[1])
lv = [x[0] for x in lv]
for i in range(1, len(lv)):
    el.add_edge(str(lv[i-1]['title']), str(lv[i]['title']))

In [ ]:
# Drawing the graph 
%matplotlib inline
nx.draw_kamada_kawai(el, with_labels=True)

In [ ]:
# Generating NewsChain event line 

# Defining similarity between sub-events 
# Similarity = avg_pairwise_eos 
# Rules for drawing edge, if sim > theta, draw directed edge based on min time 
el.clear()
